In [1]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import cv2
import os
import matplotlib.pyplot as plt
import requests
import base64
import json
import csv

In [2]:
train_dir = r'D:\AI\PART2\Cars Dataset\train'
test_dir = r'D:\AI\PART2\Cars Dataset\test'
Dataset = r'D:\AI\PART2\Cars Dataset'



In [3]:
data_set = os.listdir(Dataset)
print("รายชื่อไฟล์ในโฟลเดอร์ test:", data_set)

# แสดงรายชื่อไฟล์ในโฟลเดอร์ test
test_files = os.listdir(test_dir)
print("รายชื่อไฟล์ในโฟลเดอร์ test:", test_files)

# แสดงรายชื่อไฟล์ในโฟลเดอร์ train
train_files = os.listdir(train_dir)
print("รายชื่อไฟล์ในโฟลเดอร์ train:", train_files)


รายชื่อไฟล์ในโฟลเดอร์ test: ['test', 'train']
รายชื่อไฟล์ในโฟลเดอร์ test: ['Audi', 'Hyundai Creta', 'Mahindra Scorpio', 'Rolls Royce', 'Swift', 'Tata Safari', 'Toyota Innova']
รายชื่อไฟล์ในโฟลเดอร์ train: ['Audi', 'Hyundai Creta', 'Mahindra Scorpio', 'Rolls Royce', 'Swift', 'Tata Safari', 'Toyota Innova']


In [4]:
def setNumber_brand(b):
    if b=='Audi':
        return 1
    elif b=='Hyundai Creta':
        return 2
    elif b=='Mahindra Scorpio':
        return 3
    elif b=='Rolls Royce':
        return 4
    elif b=='Swift':
        return 5
    elif b=='Tata Safari':
        return 6
    elif b=='Toyota Innova':
        return 7

In [5]:
def numeric_data(hog_data):        
    ss = hog_data.strip('[,]')  
    sss = ss.split(', ') 
    numeric_data = [float(value) for value in sss]
    return numeric_data

In [6]:
def GETHOG(part_dir):
    
    listData = [] # List to store training data 
    folder_path = part_dir  # Make sure to set test_dir to the appropriate path
    if not os.path.exists(folder_path):
        print("โฟลเดอร์ไม่มีอยู่")
        exit()
    Amount = 0
    for filebrand in os.listdir(folder_path):
        count = 0
      
        file_path = os.path.join(folder_path, filebrand)
        for image_filename in os.listdir(file_path):
            image_path = os.path.join(file_path, image_filename)

            with open(image_path, "rb") as image_file:
                image_data = image_file.read()
                base64_encoded = base64.b64encode(image_data).decode("utf-8")

                url = r'http://localhost:8080/api/gethog'
                data64 = {
                    "img": 'data:image/' + str(os.path.splitext(image_path)[1]) + ';base64,' + base64_encoded
                }

                headers = {"Content-Type": "application/json"}
                response = requests.get(url, json=data64, headers=headers)
                
                if response.status_code == 200:
                    count = count + 1
                    data = {
                        "image": os.path.basename(image_path),
                        "HOG":  response.json()["Hog"],
                        "Class": setNumber_brand(filebrand)
                    }
                    
                    listData.append(data) 
                else:
                    print("Error:", response.status_code)
        print(str(setNumber_brand(filebrand))+'  '+filebrand+"  "+str(count))
        Amount = Amount + count
    print(f'Amount {Amount}')
    return listData

In [7]:
def loadCSV(namesFile, listDatas):
    csv_filename = namesFile + ".csv"
    with open(csv_filename, "w", newline="") as csv_file:
        fieldnames = ["image", "HOG", "Class"]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for data in listDatas:
            # Convert the "HOG" data from a list of numbers to a string
            hog_data = ', '.join(str(number) for number in data["HOG"])
            # Write the data to the CSV file
            writer.writerow({"image": data["image"], "HOG": hog_data, "Class": data["Class"]})

In [10]:
list_train = GETHOG(train_dir)

1  Audi  814
2  Hyundai Creta  271
3  Mahindra Scorpio  316
4  Rolls Royce  311
5  Swift  424
6  Tata Safari  441
7  Toyota Innova  775
Amount 3352


In [11]:
list_test = GETHOG(test_dir)

1  Audi  199
2  Hyundai Creta  67
3  Mahindra Scorpio  75
4  Rolls Royce  74
5  Swift  102
6  Tata Safari  106
7  Toyota Innova  190
Amount 813


In [14]:
loadCSV('train_data',list_train)
loadCSV('test_data',list_test)

In [15]:
data_path_train = f'D:\AI\PART2/train_data.csv'
data_path_test = f'D:\AI\PART2/test_data.csv'

In [49]:
data_train = pd.read_csv(data_path_train)
# data_train.head()
data_train.shape

(3352, 3)

In [17]:
data_test = pd.read_csv(data_path_test)
data_test.head()

,image,HOG,Class
0,1000.jpg,"0.011094658635556698, 0.012741324491798878, 0....",1
1,1003.jpg,"0.04191439598798752, 0.06864418834447861, 0.09...",1
2,1008.jpg,"0.08801346272230148, 0.021987106651067734, 0.0...",1
3,1020.jpg,"0.0021135981660336256, 0.05653822049498558, 0....",1
4,1022.jpg,"0.16168105602264404, 0.13524803519248962, 0.15...",1


***sklearn***

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn import tree

In [22]:
X_train = data_train['HOG']
Y_train = data_train['Class']
print(Y_train.unique())

[1 2 3 4 5 6 7]


In [24]:
X_test = data_test['HOG']
Y_test = data_test['Class']
print(Y_test.unique())

[1 2 3 4 5 6 7]


In [25]:
def convert_hog_string_to_vector(hog_string):
    hog_list = eval(hog_string)
    hog_vector = np.array(hog_list)
    return hog_vector

In [26]:
X_train_numeric = np.array([convert_hog_string_to_vector(hog_string) for hog_string in X_train])
X_test_numeric = np.array([convert_hog_string_to_vector(hog_string) for hog_string in X_test])

In [27]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train_numeric,Y_train)

In [28]:
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import  accuracy_score   
y_model_prediction = clf.predict(X_test_numeric)

print(accuracy_score(Y_test,y_model_prediction)*100)
print(confusion_matrix(Y_test,y_model_prediction))

46.86346863468634
[[ 74  20  18  22  15  19  31]
 [ 10  36   3   5   2   5   6]
 [ 10   6  27   4   5   5  18]
 [ 33   5   5   8   5  10   8]
 [ 16   8   4   8  44   5  17]
 [  9   8   6   4   6  69   4]
 [ 19  11   7   9   9  12 123]]


In [42]:
y_pred = clf.predict(X_test_numeric)
print("Accuracy:",metrics.accuracy_score(Y_test, y_pred))

Accuracy: 0.46863468634686345


In [35]:
import pickle

In [36]:
path_model =f'D:\AI\PART2\imageCAR_model.pkl'
pickle.dump(clf, open(path_model,'wb'))

In [29]:
import joblib

In [30]:
model_filename = 'image_model.pkl'
joblib.dump(clf, model_filename)

['image_model.pkl']